In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time

In [2]:
dog_embeddings = np.load('dog_pairs_cumulative/cumulative_embedding_output_dogs.npy')[:10000,:]

dog_subembeddings = np.load('dog_pairs_cumulative/cumulative_sub_embedding_output_dogs.npy')[:10000,:]

# compute 90th percentile thresholds between all embeddings and sub-embeddings

In [3]:
# get embedding percentile thresholds
embeddings_for_percentile_comp= np.squeeze(np.load('alexnet_imgnet_embeddings_prelu_noise.npy'))

sorted_indices_embedding = np.argsort(embeddings_for_percentile_comp,axis = 0)

indices_90_embedding = sorted_indices_embedding[900,:]

embedding_threshold_90 = embeddings_for_percentile_comp[indices_90_embedding, range(4096)].reshape(1,4096)


# get sub-embedding percentile thresholds
sub_embeddings_for_percentile_comp = np.squeeze(np.load('alexnet_imgnet_sub_embeddings_prelu_noise.npy'))

sorted_indices_sub_embedding = np.argsort(sub_embeddings_for_percentile_comp,axis = 0)

indices_90_sub_embedding = sorted_indices_sub_embedding[900,:]

sub_embedding_threshold_90 = sub_embeddings_for_percentile_comp[indices_90_sub_embedding, range(4096)].reshape(1,4096)


# compute number of co-active features between all pairs of embeddings

In [ ]:
t1 = time.time()

dog_emb_active = (dog_embeddings >= embedding_threshold_90)*1

dog_emb_active_t = dog_emb_active.T

co_active_features_dog_emb = np.matmul(dog_emb_active,dog_emb_active_t)

co_active_features_dog_emb_sorted = np.argsort(-co_active_features_dog_emb,axis = 1)

t2 = time.time()

print('time taken ='+str((t2-t1)*1000)+' seconds')

# compute number of co-active features between all pairs of sub-embeddings

In [ ]:
dog_subemb_active = (dog_subembeddings >= sub_embedding_threshold_90)*1

dog_subemb_active_t = dog_subemb_active.T

co_active_features_dog_subemb = np.matmul(dog_subemb_active,dog_subemb_active_t)

co_active_features_dog_subemb_sorted = np.argsort(-co_active_features_dog_subemb,axis = 1)

print('time taken ='+str((t2-t1)*1000)+' seconds')

# compute cosine similarity between all pairs of embeddings

In [ ]:
dog_embeddings_norm = (dog_embeddings/np.sqrt(np.sum(dog_embeddings*dog_embeddings, axis=1)).reshape(-1,1))

dog_embeddings_norm_t = dog_embeddings_norm.T

cos_sims = np.matmul(dog_embeddings_norm,dog_embeddings_norm_t)

cos_sims_sorted = np.argsort(-cos_sims,axis = 1)

# compute cosine similarity between all pairs of sub-embeddings

In [ ]:
dog_subembeddings_norm = (dog_subembeddings/np.sqrt(np.sum(dog_subembeddings*dog_subembeddings, axis=1)).reshape(-1,1))

dog_subembeddings_norm_t = dog_subembeddings_norm.T

cos_sims_sub = np.matmul(dog_subembeddings_norm,dog_subembeddings_norm_t)

cos_sims_sorted_sub = np.argsort(-cos_sims_sub,axis = 1)

# display reference image and 10 most similar images according to different metrics

In [ ]:
np.random.seed(10)
sampled_images = np.random.choice(range(10000),20,replace=False)

for main_img in sampled_images:
    img = mpimg.imread('dog_pairs_cumulative/count_'+str(main_img)+'.jpg')
    plt.figure(figsize=(5.5,2.5))
    plt.imshow(img)
    plt.axis('off')
    plt.title('reference image')
    plt.show()
    for sim in range(4):
        fig,axs = plt.subplots(1,10, figsize = (24,2.5))
        fig.tight_layout(pad=2)
        plt.subplots_adjust(top=1.0)
        if sim == 0:
            print('metric: cosine similarity between embeddings')
            sorted_array = cos_sims_sorted
            similarity_array = cos_sims
            
        elif sim == 1:
            print('metric: number of co-active embedding features (out of 4096 features)')
            sorted_array = co_active_features_dog_emb_sorted
            similarity_array = co_active_features_dog_emb

        
        elif sim == 2:
            print('metric: cosine similarity between sub-embeddings')
            sorted_array = cos_sims_sorted_sub
            similarity_array = cos_sims_sub
            
        elif sim == 3:
            print('metric: number of co-active sub-embedding features (out of 4096 features)')
            sorted_array =co_active_features_dog_subemb_sorted
            similarity_array = co_active_features_dog_subemb

            
        #plt.suptitle(title,fontsize=15)
        
        for index in range(10):
            
            img_index = sorted_array[main_img,index+1]
            img = mpimg.imread('dog_pairs_cumulative/count_'+str(img_index)+'.jpg')
            axs[index].imshow(img, aspect="auto")
            axs[index].axis('off')
            if sim == 0 or sim == 2:
                score = round(similarity_array[main_img,img_index], 2)
            else:
                score = similarity_array[main_img,img_index]
            axs[index].set_title('score = '+str(score),fontsize=20)
        plt.show()
        plt.close()